In [1]:
# Imports
import os
import csv
import spacy
from enum import Enum
from spacy import displacy
from spacy.symbols import PROPN, NOUN, CCONJ, ADP, VERB
import numpy as np
import speech_recognition as sr 
from scipy.sparse import csr_matrix
from sknetwork.path import shortest_path

from collections import defaultdict
import functools
import itertools

currentPath = os.path.dirname(os.path.abspath(''))

In [2]:
# Reading trips from csv file
# Inspired from https://stackoverflow.com/a/12398967

pathCount = 0
timeTableFileName = os.path.join(currentPath, './data/timetables_edited.csv')

# Create dictionary to associates a station name with an id
trainStationNameToId = defaultdict(functools.partial(next, itertools.count()))

with open(timeTableFileName, newline='', encoding='UTF-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')

    # Set of the reading position (ignoring header line)
    csvfile.seek(0)
    next(reader)

    # First reading to get the number different stations and init shape of trips object
    for row in reader:
        idxA = trainStationNameToId[row[1]]
        idxB = trainStationNameToId[row[2]]
    numberOfTrainstations = len(trainStationNameToId)
    trips = np.zeros((numberOfTrainstations, numberOfTrainstations))

    # Reset of the reading position (ignoring header line)
    csvfile.seek(0)
    next(reader)

    # Reading data
    for row in reader:
        pathCount += 1 
        idxA = trainStationNameToId[row[1]]
        idxB = trainStationNameToId[row[2]]

        # If trip already exist/has already be read, display message
        indexTupple = (idxA, idxB) if idxA < idxB else (idxB, idxA)
        if trips[indexTupple] != 0:
            print(f"Trip {row[1]} - {row[2]} with a distance of {row[3]} has already be read with a distance of {trips[indexTupple]}. Ignoring the new one.")
        else:
            trips[indexTupple] = int(row[3])


# Create dictionarry to map an id to its train station name
trainStationIdToName = dict((id, name) for name, id in trainStationNameToId.items())

# Make matrix symetrical as the trips are not directed but can be taken in both directions
# Source from https://stackoverflow.com/a/42209263
i_lower = np.tril_indices(numberOfTrainstations, -1)
trips[i_lower] = trips.T[i_lower]

# Creating Compressed Sparse Row (CSR) matrix to store and work more efficiently with only the trips
tripGraph = csr_matrix(trips)

print(f"Read {tripGraph.getnnz()} go and back trips ({int(tripGraph.getnnz() / 2)} distinc trips) out of {pathCount} rows for {len(trainStationNameToId)} distinct train stations.")

Read 3150 go and back trips (1575 distinc trips) out of 1575 rows for 817 distinct train stations.


In [3]:
class Trip:
    def __init__(self, startStationId, endStationId, path, totalDuration):
        self.startStationId = startStationId
        self.endStationId = endStationId
        self.path = path
        if totalDuration is None:
            self.totalDuration = None
        else:
            self.totalDuration = int(totalDuration)

    def __str__(self):
        return f"Trip from {trainStationIdToName[self.startStationId]} to {trainStationIdToName[self.endStationId]} for a total duration of {self.totalDuration} by this path: {self.path}"

In [4]:
# Functions used to determine the shortest path between cities 

def getPathBetweenIds(trainStationStartIds: list, trainStationEndIds: list):
    global tripGraph
    paths = []
    # If start array contains one element also contained in end array -> return path from/to the same station
    for startId in trainStationStartIds:
        if startId in trainStationEndIds:
            return [int(startId), int(startId)]

    # As shortest_path() does not support multiple sources and multiple targets at the same time, we'll iterate through all start points and manually concat the results
    if(len(trainStationStartIds) > 1 and len(trainStationEndIds) > 1):
        for trainStationEndId in trainStationEndIds:
            results = shortest_path(tripGraph, sources=[int(i) for i in trainStationStartIds], targets=[int(trainStationEndId)], method='D')
            for result in results:
                if len(result) >= 2:
                    paths.append(result)
        return paths
    else:
        results = shortest_path(tripGraph, sources=[int(i) for i in trainStationStartIds], targets=[int(i) for i in trainStationEndIds], method='D')
        for result in results:
            if len(result) >= 2:
                paths.append(result)
        return paths

def isCityMatchingKey(city: str, key: str):
    return city.lower() in key.lower()

def getPathBetweenCities(start: str, end: str):
    trainStationStartIds = np.array([])
    trainStationEndIds = np.array([])
    
    # Get all stations that contains the searched name
    for key, value in trainStationNameToId.items():
        if isCityMatchingKey(start, key):
            trainStationStartIds = np.append(trainStationStartIds, value)
        if isCityMatchingKey(end, key):
            trainStationEndIds = np.append(trainStationEndIds, value)

    if len(trainStationStartIds) > 0 and len(trainStationEndIds) > 0:
        return getPathBetweenIds(trainStationStartIds, trainStationEndIds)
    else:
        return np.array([])

def getBestPathForFullTrip(tripCityWaypoints: list):
    global tripGraph
    fullTrip = np.array(np.zeros(len(tripCityWaypoints)-1), dtype=object)
    # Iterate through all sub trips
    for trip in range(len(fullTrip)):
        paths = getPathBetweenCities(tripCityWaypoints[trip], tripCityWaypoints[trip+1])
        minDistance = None
        keptPath = None
        startId = None
        endId = None

        # Iterate though the returned array
        for path in paths:
            distance = 0
            # Nested array => multiple start/end possible
            if isinstance(path, list):
                for i in range(len(path)-1):
                    distance = distance + tripGraph[(path[i], path[i+1])]
                if minDistance is None or distance < minDistance:
                    minDistance = distance
                    keptPath = path
                    startId = path[0]
                    endId = path[len(path)-1]
            
            # Scalar value => only one path possible
            else:
                for i in range(len(paths)-1):
                    distance = distance + tripGraph[(paths[i], paths[i+1])]
                minDistance = distance
                keptPath = paths
                startId = paths[0]
                endId = paths[len(paths)-1]

        fullTrip[trip] = Trip(startId, endId, keptPath, minDistance)
    return fullTrip


# Use following to test pathfinding functions above
def testPathfinding():
    bestTrips = getBestPathForFullTrip(['Orléan', 'Paris', 'Strasbourg', 'dsfdsfd', 'Mulhouse', 'Mulhouse'])
    for i in range(len(bestTrips)):
        if bestTrips[i].path is not None:
            print(f"#{i+1} - {bestTrips[i]}")
            for stationId in bestTrips[i].path:
                print(f"{trainStationIdToName[stationId]}")
        else:
            if bestTrips[i].startStationId is None or bestTrips[i].endStationId is None:
                print(f"#{i+1} - Could not find one or both station of the sub-trip")
            else:
                print(f"#{i+1} - Could not find a path between the both stations")
testPathfinding()

#1 - Trip from Gare de Orléans to Gare de Paris-Austerlitz for a total duration of 75 by this path: [18, 12]
Gare de Orléans
Gare de Paris-Austerlitz
#2 - Trip from Gare de Paris-Est to Gare de Strasbourg for a total duration of 287 by this path: [735, 669, 98, 104]
Gare de Paris-Est
Gare de Epernay
Gare de Metz-Ville
Gare de Strasbourg
#3 - Could not find one or both station of the sub-trip
#4 - Could not find one or both station of the sub-trip
#5 - Trip from Gare de Mulhouse to Gare de Mulhouse for a total duration of 0 by this path: [133, 133]
Gare de Mulhouse
Gare de Mulhouse


In [5]:
# Getting user's request
# Inspired from https://www.geeksforgeeks.org/python-convert-speech-to-text-and-text-to-speech/
try: 
    r = sr.Recognizer()
    with sr.Microphone() as source: 
        print("Adjusting to noise level...")
        r.adjust_for_ambient_noise(source, duration=0.2) 
 
        print("Listening...")
        audio = r.listen(source) 
            
        print("Voice recognition...")
        request = r.recognize_google(audio, language="fr-FR") 

        print(f"Parsed: '{request}'") 
            
except sr.RequestError as e: 
    print(f"Exception during request parsing: {e}") 

Adjusting to noise level...
Listening...
Voice recognition...
Parsed: 'j'aimerais aller de Mulhouse à Paris'


In [5]:
# Extract cities list in the right travel order from the user's request

# Spacy documentation: https://spacy.io/usage/linguistic-features
# NLP Feature that we use or can be useful:
#   - Lemmatization: we use it to avoid defining every possible form of a word when defining our rules
#   - PoS Tagging: we use it to detect the function of a word which helps us to choose which rule we apply and reduce the number of checks we need to do
#   - Dependency Parsing: we use it to navigate in the sentence between the words that are linked
#   - Word Senses: we use it to define our rules that assigns a specific direction and strength to apply to certain words/fixed word groups
#   - Constituent Parsing: we currently don't use it, but it might help us improve reliability by splitting sentences into sub-sentences that we can then organize between them


# Examples:
#request = "J'aimerais aller d'Orléans à Paris puis dans les Vosges"
#request = "Je veux aller à Marseille à partir de Lyon"
request = "Je veux visiter Paris en partant de Bordeaux et en passant par Nantes"
#request = "Je veux prendre le train à Mulhouse à destination de Strasbourg"
#request = "Strasbourg en provenance de Mulhouse"
#request = "Je veux aller de Mulhouse à Strasbourg"
#request = "Je veux faire Paris Gare De l'est Marseil"
#request = "Je veux aller à Paris après être allé à Mulhouse depuis Lyon" #TODO find a way to make this work without breaking the rest as the structure is quite tricky (would Constituent Parsing help here?)
#request = "Paris Marseille" #TODO Does not pass - Issue here is that spacy does not recognize Marseille as a city, maybe try with fr_core_news_md or fr_core_news_lg (also check if the performances are still the sames)

print(f"Request: {request}")

class RelationDirection(Enum):
    NONE = 1
    START = 2
    DEST = 3

class RelationStrength(Enum):
    NONE = 1
    WEAK = 2
    STRONG = 3


class WordSense:
    def __init__(self, word: str, direction: RelationDirection, strength: RelationStrength):
        self.word = word
        self.direction = direction
        self.strength = strength

    def __str__(self):
     return f"Word '{self.word}' has a direction of {self.direction.name} and a {self.strength.name} strength."

class LinkedWordSense:
    def __init__(self, word: str, fixedWord: str, direction: RelationDirection, strength: RelationStrength):
        self.word = word
        self.fixedWord = fixedWord
        self.direction = direction
        self.strength = strength

    def __str__(self):
     return f"Words '{self.word}' fixed with '{self.fixedWord}' has a direction of {self.direction.name} and a {self.strength.name} strength."

# CCONJ links: 'cc'_child
CCONJ_Relation = [
    # Start
    WordSense("depuis",     RelationDirection.START, RelationStrength.STRONG),
    # Destination
    WordSense("puis",       RelationDirection.DEST,  RelationStrength.STRONG),
    WordSense("et",         RelationDirection.DEST,  RelationStrength.STRONG),
    WordSense("enfin",      RelationDirection.DEST,  RelationStrength.STRONG)
]

# NOUN links: 'nmod'_parent
NOUN_Relation = [
    # Start
    WordSense("provenance",     RelationDirection.START, RelationStrength.STRONG),
    # Destination
    WordSense("direction",      RelationDirection.DEST,  RelationStrength.WEAK),
    WordSense("destination",    RelationDirection.DEST,  RelationStrength.WEAK)
]

# ADP_FIXED has the priority 
# ADP links: 'case'_child, 'dep'_parent
ADP_FIXED_Relation = [
    # Start
    LinkedWordSense("à","partir",       RelationDirection.START, RelationStrength.STRONG),
    LinkedWordSense("en", "partant",    RelationDirection.START, RelationStrength.STRONG),
    # Destination
    LinkedWordSense("à","destination",  RelationDirection.DEST,  RelationStrength.STRONG),
    LinkedWordSense("en","direction",   RelationDirection.DEST,  RelationStrength.WEAK)
]
ADP_Relation = [
    # Start
    WordSense("de",     RelationDirection.START, RelationStrength.STRONG),
    WordSense("du",     RelationDirection.START, RelationStrength.STRONG),
    WordSense("des",    RelationDirection.START, RelationStrength.STRONG),
    # Destination
    WordSense("à",      RelationDirection.DEST,  RelationStrength.WEAK),
    WordSense("au",     RelationDirection.DEST,  RelationStrength.WEAK),
    WordSense("aux",    RelationDirection.DEST,  RelationStrength.WEAK),
    WordSense("dans",   RelationDirection.DEST,  RelationStrength.WEAK),
    WordSense("en",     RelationDirection.DEST,  RelationStrength.WEAK),
    WordSense("par",    RelationDirection.DEST,  RelationStrength.WEAK) # par : "passer par Paris"
] 

# VERB links: 'obl:arg'_parent, 'obl:mod'_parent
# "partir" is ambiguous: "partir de ..." "partir à ..."
VERB_Relation = [
    # Start
    WordSense("décoller",   RelationDirection.START, RelationStrength.STRONG),
    WordSense("passer",   RelationDirection.START, RelationStrength.WEAK),
    # Destination
    WordSense("arriver",    RelationDirection.DEST,  RelationStrength.STRONG),
    WordSense("aller",      RelationDirection.DEST,  RelationStrength.STRONG),
    WordSense("visiter",    RelationDirection.DEST,  RelationStrength.STRONG),
    WordSense("atterrir",   RelationDirection.DEST,  RelationStrength.STRONG),
    WordSense("découvrir",  RelationDirection.DEST,  RelationStrength.STRONG),
    WordSense("voyager",    RelationDirection.DEST,  RelationStrength.STRONG),
    WordSense("rendre",     RelationDirection.DEST,  RelationStrength.STRONG)
]

nlp = spacy.load("fr_core_news_sm")
doc = nlp(request)
locations = []
fullTrip = []

# Extract locations
for i in doc.ents:
    if i.label_ == 'LOC' or i.label_ == 'GPE': 
        locations.append(i.text)
print(f"Locations found: {locations}")

if len(locations) == 0:
    print("Sorry, but we cannot answer this request.")
else:
    # Get token for each locations
    tokens = np.zeros(len(locations), dtype=object)
    for i in range(len(locations)):
        tokenFound = False
        # Priority: PROPN
        for token in doc:
            if token.pos == PROPN:
                isUsable = True
                for tokenSelected in tokens:
                    if type(tokenSelected) != int and tokenSelected == token:
                        isUsable = False
                if isUsable:
                    if token.text in locations[i]:
                        tokens[i] = token
                        tokenFound = True
                        break

        # Secondary: NOUN
        if tokenFound == False:
             for token in doc:
                if token.pos == NOUN:
                    isUsable = True
                    for tokenSelected in tokens:
                        if type(tokenSelected) != int and tokenSelected == token:
                            isUsable = False
                    if isUsable:
                        if token.text in locations[i]:
                            tokens[i] = token
                            tokenFound = True
                            break

        # Failsafe: any (e.g in "Je veux faire Paris Gare De l'Est Marseil" marseille is parsed as a VERB)
        if tokenFound == False:
            for token in doc:
                isUsable = True
                for tokenSelected in tokens:
                    if type(tokenSelected) != int and tokenSelected == token:
                        isUsable = False
                if isUsable:
                    if token.text in locations[i]:
                        tokens[i] = token
                        tokenFound = True
                        break

        # None
        if tokenFound == False:
            print(f"Localization {locations[i]} not found")
            tokens[i] = None

    # Remove None tokens
    tmpTokens = tokens
    tokens = [] 
    for token in tmpTokens: 
        if token != None : 
            tokens.append(token)


    # Weight tokens to prepare ordering
    weighedTokens = np.zeros(len(tokens), dtype=object)
    for i in range(len(tokens)):

        # If any CCONJ => use it
        cconjFound = False
        for child in tokens[i].children:
            if cconjFound == True:
                break
            if child.pos == CCONJ:
                for ref in CCONJ_Relation:
                    if ref.word == child.lemma_:
                        cconjFound = True
                        weighedTokens[i] = (tokens[i], ref)
                        break

        # Else try NOUN
        if cconjFound == False:
            nounFound = False
            parent = tokens[i].head
            if parent.pos == NOUN:
                for ref in NOUN_Relation:
                    if ref.word == parent.lemma_:
                        nounFound = True
                        weighedTokens[i] = (tokens[i], ref)
                        break

            # Else try ADP
            if nounFound == False:
                adpFound = False
                # If any ADP_FIXED => use it
                for child in tokens[i].children:
                    if adpFound == True:
                        break
                    if child.pos == ADP:
                        for subChild in child.children:
                            if subChild.dep_ == 'fixed':
                                for ref in ADP_FIXED_Relation:
                                    if ref.word == child.lemma_ and ref.fixedWord == subChild.lemma_:
                                        adpFound = True
                                        weighedTokens[i] = (tokens[i], ref)
                                        break
                # Else if any ADP => use it
                if adpFound == False:
                    for child in tokens[i].children:
                        if adpFound == True:
                            break
                        for ref in ADP_Relation:
                            if ref.word == child.lemma_:
                                adpFound = True
                                weighedTokens[i] = (tokens[i], ref)
                                break
            
                # Else if any VERB => use it
                if adpFound == False:
                    verbFound = False
                    parent = tokens[i].head
                    if parent.pos == VERB:
                        for ref in VERB_Relation:
                            if ref.word == parent.lemma_:
                                verbFound = True
                                weighedTokens[i] = (tokens[i], ref)
                                break

                    # Else Keep position 
                    if verbFound == False:
                        weighedTokens[i] = (tokens[i], WordSense("", RelationDirection.DEST,  RelationStrength.WEAK))
    

    # Order tokens
    orderedTokens = []
    # First pass for direction: START
    for i in range(len(weighedTokens)):
        token, weight = weighedTokens[i]
        numberOfStrongStrength = 0
        if weight.direction == RelationDirection.START:
            if weight.strength == RelationStrength.STRONG:
                orderedTokens.insert(numberOfStrongStrength, token)
                numberOfStrongStrength = numberOfStrongStrength + 1
            else:
                orderedTokens.append(token)
    

    # Second pass for direction: DEST
    for i in range(len(weighedTokens)):
        token, weight = weighedTokens[i]
        numberOfStrongStrength = 0
        if weight.direction == RelationDirection.DEST:
            if weight.strength == RelationStrength.STRONG:
                orderedTokens.append(token)
                numberOfStrongStrength = numberOfStrongStrength + 1
            else:
                if numberOfStrongStrength == 0:
                    orderedTokens.append(token)
                else:
                    orderedTokens.insert(len(numberOfStrongStrength)-numberOfStrongStrength, token)

    # Populate full trip cities list
    for token in orderedTokens:
        fullTrip.append(token.text)
    print(f"Result trip: {fullTrip}")

# DEBUG
#for token in doc:
#    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

#displacy.serve(doc, style="dep")


Request: Je veux visiter Paris en partant de Bordeaux et en passant par Nantes
Locations found: ['Paris', 'Bordeaux', 'Nantes']
Result trip: ['Bordeaux', 'Paris', 'Nantes']


In [6]:
resultTrips = getBestPathForFullTrip(fullTrip)
for i in range(len(resultTrips)):
    if resultTrips[i].path is not None:
        print(f"#{i+1} - {resultTrips[i]}")
        for stationId in resultTrips[i].path:
            print(f"{trainStationIdToName[stationId]}")
    else:
        if resultTrips[i].startStationId is None or resultTrips[i].endStationId is None:
            print(f"#{i+1} - Could not find one or both station of the sub-trip")
        else:
            print(f"#{i+1} - Could not find a path between the both stations")

#1 - Trip from Gare de Bordeaux-St-Jean to Gare de Paris-Austerlitz for a total duration of 375 by this path: [605, 676, 13, 12]
Gare de Bordeaux-St-Jean
Gare de Limoges-Bénédictins
Gare de Vierzon
Gare de Paris-Austerlitz
#2 - Trip from Gare de Paris-Austerlitz to Gare de Nantes for a total duration of 216 by this path: [12, 21, 19, 408]
Gare de Paris-Austerlitz
Gare de St-Pierre-des-Corps
Gare de Tours
Gare de Nantes
